# 非聚合类方法
- 非聚合指的是数据处理前后没有进行分组操作，数据列的长度没有发生改变

## map()
- map()针对单列Series进行处理
- map()还有一个参数na_action，取值为None或ingore，用于控制遇到缺失值的处理方式。设置为ingore时串行运算过程中将忽略Nan值原样返回。

### 字典映射

In [ ]:
gender2xb = {'F': '女性', 'M': '男性'}

#利用map()方法得到对应gender列的映射列
data.gender.map(gender2xb)

### lambda函数

In [ ]:
data.gender.map(lambda x:'女性'if x is'F'else'男性')

### 常规函数

In [ ]:
def gender_to_xb(x):

    return '女性' if x is 'F' else '男性'

data.gender.map(gender_to_xb)

### 规范化输出

In [ ]:
data.gender.map("This kid's gender is {}".format)

****——————————————————————————————————————————————————****

## apply()
- apply()堪称pandas中最好用的方法,可以对单列或多列进行运算，覆盖非常多的使用场景。

### 结合lambda实现单列操作

In [ ]:
# 此处同map()
data.gender.apply(lambda x:'女性'if x is'F'else'男性')

### 多列操作

#### 多列输入

In [ ]:
def generate_descriptive_statement(year, name, gender, count):
    year, count = str(year), str(count)
    gender = '女性' if gender is 'F' else '男性'

    return '在{}年，叫做{}性别为{}的新生儿有{}个。'.format(year, name, gender, count)


data.apply(lambda row:generate_descriptive_statement(row['year'],
                                                      row['name'],
                                                      row['gender'],
                                                      row['count']),
                                                      axis = 1   # 按行运行
          )

#### 多列输出

In [ ]:
# 利用apply()来提取name列中的首字母和剩余部分字母
data.apply(lambda row: (row['name'][0], row['name'][1:]), axis=1)  # zip(*zipped)可以用来解开元组序列
# 返回的是单列结果，每个元素是返回值组成的元组

### 显示进度条

In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook

tqdm_notebook.pandas(desc='apply')
data.progress_apply(lambda row:generate_descriptive_statement(row['year'],
                                                      row['name'],
                                                      row['gender'],
                                                      row['count']),
                                                      axis = 1
                   )

****——————————————————————————————————————————————————****

## applymap()
- applymap()将传入的函数等作用于整个数据框中每一个位置的元素，因此其返回结果的形状与原数据框一致

In [ ]:
def lower_all_string(x):  
    if isinstance(x, str):  # isinstance()用于判断是否为某一类型数据
        return x.lower()
    else:
        return x

data.applymap(lower_all_string)  # applymap()在全部数据上运行函数，不改变原数据结构，用于批量数据处理

****——————————————————————————————————————————————————****

# 聚合类操作
- 像SQL里的聚合操作那样将原始数据分组运算

## groupby()

In [ ]:
#按照年份和性别对婴儿姓名数据进行分组
groups = data.groupby(by=['year','gender'])

#利用列表解析提取分组结果
groups = [group for group in groups]

# 查看其中的一个元素：
groupso[0]

### 直接调用聚合函数
- (.reset_index(drop=False)将索引还原回数据框)

In [ ]:
#求每个分组中最高频次  
data.groupby(by=['year','gender'])['count'].max().reset_index(drop=False)

### 结合apply()

In [ ]:
import numpy as np

def find_most_name(df):
    return str(np.max(df['count']))+'-'+df['name'][np.argmax(df['count'])]

data.groupby(['year','gender']).apply(find_most_name).reset_index(drop=False)

### 利用agg()进行更灵活的聚合
- agg即aggregate，聚合

#### 聚合单列

In [ ]:
#求count列的最小值、最大值以及中位数
data['count'].agg(['min','max','median'])

#### 聚合数据框
- 对数据框进行聚合时因为有多列，所以要使用字典的方式传入聚合方案

In [ ]:
 data.agg({'year': ['max','min'], 'count': ['mean','std']})

#### 聚合groupby()结果
- .reset_index(drop=False)还原索引列
- pd.NamedAgg()还原聚合运算列

In [ ]:
data.groupby(['year','gender']).agg(
    min_count=pd.NamedAgg(column='count', aggfunc='min'),
    max_count=pd.NamedAgg(column='count', aggfunc='max'),
    median=pd.NamedAgg(column='count', aggfunc='median')).reset_index(drop=False)